# Embedding Agent

What if I trained an agent on word embeddings through reinforcement learning, to predict the results of analogies à la "king - man + woman = queen"?

Can I study the world model it builds, and thus deduce things about the world models that could be built from the semantic embeddings?
Can I apply IRL methodology to study the reward structure?
Can I use the agents knowledge of semantics to have it act in a (ad hoc and virtual) world?

**WON'T THE AGENT SIMPLY LEARN TO RECONSTRUCT THE EMBEDDING SPACE (OR AT LEAST LOCAL PARTS OF IT)? THIS EXPERIMENT NEEDS MORE CONSIDERATION: AS IT STANDS IT'S JUST NOT AS INTERESTING AS I FIRST IMAGINED IT WOULD BE...**

(To download the glove embeddings, go here: https://nlp.stanford.edu/projects/glove/)

## Imports

In [1]:
import os
import random

In [2]:
from gensim.models import KeyedVectors

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

import spacy

## Embedding space investigations

In [3]:
nlp = spacy.load("en_core_web_md")

glove_dir = "glove6B"
glove_file = os.path.join(glove_dir, "glove.6B.100d.txt")

In [4]:
embeddings_dict = {}
with open(glove_file, 'r', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        
        token = values[0]
        embeddings = list(map(float, values[1:]))
        
        embeddings_dict[token] = embeddings

In [5]:
# get parameter count and embedding dimension
_, pc, ed, _ = glove_file.split(".")
path_w2v = os.path.abspath(f'glove6B/glove.{pc}.{ed}.w2v.txt')

glove = datapath(glove_file)
w2v = get_tmpfile(path_w2v)

model = KeyedVectors.load_word2vec_format(w2v, binary=False)

In [6]:
nouns = []
for token in list(embeddings_dict.keys())[:3_000]:
    doc = nlp(token)
    for token in doc:
        if token.pos_ == 'NOUN':
            nouns.append(token.text)

In [7]:
noun_set = set(nouns)
# similarity threshold (controls the quality of the data)
threshold = 0.7

In [16]:
model.similarity("boy", "woman")

0.77205104

In [21]:
model.similar_by_vector(model["go"] - model["going"] + model["seeing"])

[('seeing', 0.9027366042137146),
 ('see', 0.8243426084518433),
 ('everyone', 0.7870626449584961),
 ('waiting', 0.7580298185348511),
 ('watching', 0.7564365267753601),
 ('looking', 0.7529377937316895),
 ('come', 0.7498775124549866),
 ('them', 0.7472339272499084),
 ('find', 0.7459867000579834),
 ('why', 0.7415868639945984)]

Generate analogies along the lines of " `w1` is to `w2` as `x` is to `w3` ".

In [39]:
w1 = random.choice(list(noun_set))
w2 = random.choice(list(noun_set - {w1}))
w3 = random.choice(list(noun_set - {w1, w2}))

result = model[w1] - model[w2] + model[w3]

xs = model.similar_by_vector(result, topn=5)

# TODO this needs more robuts checking to make sure I don't repeat words
analogy = (w1, w2, xs[0][0], w3) if xs[0][0] not in {w1, w2, w3} else (w1, w2, xs[1][0], w3)

In [40]:
analogy

('administration', 'progress', 'lynch', 'analyst')